# Recommender Systems 2024/25

### Practice 9 - Deep Learning Models

## The basics of Deep Learning: Multi-Layer Perceptron 

In [1]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()

URM_train_val, URM_test = split_train_in_two_percentage_global_sample(URM_all, 0.8)
URM_train, URM_val = split_train_in_two_percentage_global_sample(URM_train_val, 0.8)

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: Movielens10M
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10106, feature occurrences: 106820, density 9.90E-04
	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10126, feature occurrences: 128384, density 1.19E-03
	ICM name: ICM_year, Value range: 1.92E+03 / 2.01E+03, Num features: 1, feature occurrences: 10681, density 1.00E+00




In [2]:
# Training and testing
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_test = EvaluatorHoldout(URM_test, [10])
evaluator_validation = EvaluatorHoldout(URM_val, [10])

EvaluatorHoldout: Ignoring 77 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 266 ( 0.4%) Users that have less than 1 test interactions


In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

if torch.backends.mps.is_available(): # if torch.cuda.is_available() if you use NVIDIA GPUs
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [4]:
# load iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create a custom dataset class
class IrisDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
# create a custom nn.Module class
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# create a data loader and model
dataset = IrisDataset(X_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
model = MLP()

# define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# train the model
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print('Epoch %d, loss: %.3f' % (epoch+1, running_loss/(i+1)))

# evaluate the model
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data in data_loader:
        inputs, labels = data
        outputs = model(inputs)
        test_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()

accuracy = correct / len(dataset)
print('Test loss: {:.3f}, Accuracy: {:.2f}'.format(test_loss/(len(data_loader)), accuracy*100))

Epoch 1, loss: 1.118
Epoch 2, loss: 1.091
Epoch 3, loss: 1.075
Epoch 4, loss: 1.060
Epoch 5, loss: 1.046
Epoch 6, loss: 1.038
Epoch 7, loss: 1.020
Epoch 8, loss: 1.008
Epoch 9, loss: 0.997
Epoch 10, loss: 0.988
Epoch 11, loss: 0.973
Epoch 12, loss: 0.957
Epoch 13, loss: 0.937
Epoch 14, loss: 0.919
Epoch 15, loss: 0.898
Epoch 16, loss: 0.872
Epoch 17, loss: 0.852
Epoch 18, loss: 0.831
Epoch 19, loss: 0.816
Epoch 20, loss: 0.798
Epoch 21, loss: 0.786
Epoch 22, loss: 0.766
Epoch 23, loss: 0.751
Epoch 24, loss: 0.734
Epoch 25, loss: 0.717
Epoch 26, loss: 0.704
Epoch 27, loss: 0.700
Epoch 28, loss: 0.678
Epoch 29, loss: 0.664
Epoch 30, loss: 0.651
Epoch 31, loss: 0.642
Epoch 32, loss: 0.624
Epoch 33, loss: 0.617
Epoch 34, loss: 0.605
Epoch 35, loss: 0.594
Epoch 36, loss: 0.582
Epoch 37, loss: 0.578
Epoch 38, loss: 0.568
Epoch 39, loss: 0.560
Epoch 40, loss: 0.549
Epoch 41, loss: 0.542
Epoch 42, loss: 0.533
Epoch 43, loss: 0.526
Epoch 44, loss: 0.514
Epoch 45, loss: 0.512
Epoch 46, loss: 0.5

In [4]:
import numpy as np
import scipy.sparse as sp

In [8]:
from Recommenders.BaseRecommender import BaseRecommender

class DeepLearningRecommender(nn.Module, BaseRecommender):

    def __init__(self, URM_train, verbose=True):
        super().__init__()
        BaseRecommender.__init__(self, URM_train, verbose)
        self.device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

    def _data_generator(self, batch_size, num_negatives=3, num_items=None):
        user_input, item_input, labels = [], [], []
        dok_train = URM_train.todok() # <- Dictionary representation of a sparse matrix: allows us to check existing interactions as key-value pairs
        if num_items is None : num_items = self.URM_train.shape[1]

        self.batch_counter = 0
        start = self.batch_counter
        stop = min(self.batch_counter + batch_size, len(dok_train.keys()))
        for (u,i) in dok_train[start:stop].keys(): # TODO: Too many interactions batched together. Fix start:stop
            # positive interaction
            user_input.append(u)
            item_input.append(i)
            labels.append(1) # <- (Implicit ratings)
            # negative interactions
            for t in range(num_negatives): # <- num_negatives is a hyperparameter
                # randomly select an interaction; check if negative
                j = np.random.randint(num_items)
                while (u,j) in dok_train:
                    j = np.random.randint(num_items)
                user_input.append(u)
                item_input.append(j)
                labels.append(0)
        self.batch_counter += 1
        
        user_input = torch.tensor(user_input, dtype=torch.int32, device=self.device)
        item_input = torch.tensor(item_input, dtype=torch.int32, device=self.device)
        labels = torch.tensor(labels, dtype=torch.int32, device=self.device)
        labels = labels.reshape((labels.shape[0],1))
        yield user_input, item_input, labels
    
    def forward(self, user_input, item_input=None):
        raise NotImplementedError("Forward function not implemented.")

    def fit(self, epochs=30, batch_size=1024, learning_rate=0.0001):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate) # <- The optimizer can be (additionally) considered as a hyperparameter
        for i in range(epochs):
            for user_input, item_input, labels in self._data_generator(batch_size):
                optimizer.zero_grad()
                predictions = self.forward(user_input, item_input)
                loss = torch.nn.BCELoss().to(self.device) # <- The loss function can be (additionally) considered as a hyperparameter
                loss = loss(predictions, labels.float())
                loss.backward()
                optimizer.step()
            self._print("Epoch {} finished. Loss: {}".format(i, loss.item()))

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        step = user_id_array.shape[0]
        
        if items_to_compute is None:
            items_to_compute = np.arange(self.URM_train.shape[1], dtype=np.int32)
        
        predictions = np.empty((step,items_to_compute.shape[0]))
        for item in items_to_compute: # <- Parallelization is done by batches of users (could be sub-optimal?)
            with torch.no_grad():
                predictions[:, item] = self.forward(
                    torch.tensor(user_id_array),
                    torch.tensor(
                        np.ones(step, dtype=np.int32) * item)
                    ).cpu().detach().numpy().ravel()
        return predictions
    

## AutoEncoders

### Denoising Autoencoder

In [9]:
class DenoisingAutoencoder(DeepLearningRecommender):

    RECOMMENDER_NAME = """DENOISING_AUTOENCODER"""
    def __init__(self, URM_train, encoding_dim=69, noise_p=0.1, verbose=True):
        super().__init__(URM_train, verbose)
        self.noise_p = noise_p
        num_items = URM_train.shape[1]
        self.encoder = nn.Sequential(
            nn.Linear(num_items, 420),
            nn.ReLU(),
            nn.Linear(420, encoding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 420),
            nn.ReLU(),
            nn.Linear(420, num_items)
        )
        self.to(self.device)
    
    # override: both input and label are batches of user profiles
    def _data_generator(self, batch_size):
        row_idx = np.arange(self.URM_train.shape[0])
        for start in range(0, len(row_idx), batch_size):
            end = min(len(row_idx), start + batch_size)
            user_input = torch.tensor(self.URM_train[row_idx[start:end],:].toarray(), dtype=torch.float32, device=self.device)
            labels = user_input
            yield user_input, _, labels

    def forward(self, user_input, item_input=None):
        # assert(item_input == None, "Item input not needed")
        noisy_input = self._add_noise(user_input)
        encoded = self.encoder(noisy_input)
        reconstructed = self.decoder(encoded)
        return reconstructed

    # override: evaluator passes user profile ids as inputs, we need the
    #           full profiles for the forward function to work properly
    def _compute_item_score(self, user_id_array, items_to_compute=None):
        user_profiles = self.URM_train[user_id_array, :]

        if items_to_compute is not None:
            mask = np.zeros(self.URM.shape[1], dtype=np.int32)
            mask[items_to_compute] = 1
            user_profiles = user_profiles[:, mask]

        with torch.no_grad():
            predictions = self.forward(torch.tensor(user_profiles.toarray(), dtype=torch.float32, device=self.device))

        return predictions.cpu().detach().numpy()

    def _add_noise(self, x):
        zeros_mask = np.random.choice([False,True], size=x.shape, p=[1-self.noise_p, self.noise_p])
        ones_mask = np.random.choice([False,True], size=x.shape, p=[self.noise_p, 1-self.noise_p])
        x[zeros_mask] = 0
        x[ones_mask] = 1
        return x

In [10]:
denoising_autoencoder = DenoisingAutoencoder(URM_train)

denoising_autoencoder.fit(epochs=100, batch_size=1024, learning_rate=0.01)

results_df, _ = evaluator_test.evaluateRecommender(denoising_autoencoder)

results_df

DENOISING_AUTOENCODER: URM Detected 76 ( 0.7%) items with no interactions.
DENOISING_AUTOENCODER: Epoch 0 finished. Loss: 5.801004409790039
DENOISING_AUTOENCODER: Epoch 1 finished. Loss: 6.98549222946167
DENOISING_AUTOENCODER: Epoch 2 finished. Loss: 1.5783166885375977
DENOISING_AUTOENCODER: Epoch 3 finished. Loss: 1.5718098878860474
DENOISING_AUTOENCODER: Epoch 4 finished. Loss: 1.5724763870239258
DENOISING_AUTOENCODER: Epoch 5 finished. Loss: 2.0860402584075928
DENOISING_AUTOENCODER: Epoch 6 finished. Loss: 0.6773252487182617
DENOISING_AUTOENCODER: Epoch 7 finished. Loss: 0.5001788139343262
DENOISING_AUTOENCODER: Epoch 8 finished. Loss: 0.5002332925796509
DENOISING_AUTOENCODER: Epoch 9 finished. Loss: 90.19352722167969
EvaluatorHoldout: Processed 69803 (100.0%) in 43.50 sec. Users per second: 1605


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.004884,0.005148,0.001702,0.001202,0.001255,0.011424,0.002684,0.002525,0.047362,0.011707,...,0.998927,0.047311,0.998927,0.001035,3.39412,0.903482,0.005317,0.29972,0.157741,0.147761


### $EASE^R$

In [11]:
from sklearn.preprocessing import normalize

def fit(self, topK=None, l2_norm = 1e3, normalize_matrix = False):

        if normalize_matrix:
            # Normalize rows and then columns
            self.URM_train = normalize(self.URM_train, norm='l2', axis=1)
            self.URM_train = normalize(self.URM_train, norm='l2', axis=0)
            self.URM_train = sp.csr_matrix(self.URM_train)


        # Grahm matrix is X^t X, compute dot product
        grahm_matrix = self.URM_train.T.dot(self.URM_train).toarray()

        diag_indices = np.diag_indices(grahm_matrix.shape[0])
        grahm_matrix[diag_indices] += l2_norm

        P = np.linalg.inv(grahm_matrix)

        B = P / (-np.diag(P))

        B[diag_indices] = 0.0

In [6]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

model = EASE_R_Recommender(URM_train)

model.fit() # <- hyperparams left to default value, obviously could (and should) be optimized

results_df, _ = evaluator_test.evaluateRecommender(model)

results_df

EASE_R_Recommender: URM Detected 68 ( 0.6%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 16.00 sec
EvaluatorHoldout: Processed 69801 (100.0%) in 21.88 sec. Users per second: 3190


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.27802,0.32341,0.189994,0.166262,0.189054,0.539328,0.279971,0.225729,0.870288,0.925778,...,0.998898,0.869329,0.998898,0.025356,8.333874,0.994708,0.130267,0.735921,1.688469,0.091234


## Two-Tower Models

In [12]:
# Variant 1:
# 1. Make 2 embeddings of equal dimensions and concatenate
# 2. Couple of Dense layers
# 3. Obtain prediction (single score)
class TwoTowerRecommender_type1(DeepLearningRecommender):
    
    RECOMMENDER_NAME = """TWO_TOWER_1"""
    
    def __init__(self, URM_train, num_users, num_items, layers=[10], reg_layers=[0], verbose = True):
        super().__init__(URM_train, verbose)
        self.mlp_embedding_user = nn.Embedding(num_users, int(layers[0]/2), device=self.device)
        self.mlp_embedding_item = nn.Embedding(num_items, int(layers[0]/2), device=self.device)

        self.mlp_layers = nn.ModuleList([
            nn.Linear(layers[i-1], layers[i], bias=True, device=self.device) for i in range(1, len(layers))
            ])
        for i, layer in enumerate(self.mlp_layers):
            nn.init.normal_(layer.weight)
            layer.bias.data.zero_()
            layer.weight_decay = reg_layers[i]

        self.prediction_layer = nn.Linear(layers[-1], 1, bias=True, device=self.device)
        nn.init.uniform_(self.prediction_layer.weight)
        self.prediction_layer.bias.data.zero_()
        self.to(self.device)

    def forward(self, user_input, item_input):
        mlp_user_latent = self.mlp_embedding_user(user_input.long().to(self.device))
        mlp_item_latent = self.mlp_embedding_item(item_input.long().to(self.device))
        mlp_vector = torch.cat((mlp_user_latent, mlp_item_latent), dim=1)
        for layer in self.mlp_layers:
            mlp_vector = torch.relu(layer(mlp_vector))

        predict_vector = mlp_vector
        prediction = torch.sigmoid(self.prediction_layer(predict_vector))
        return prediction

In [13]:
# Variant 2:
# 1. Couple of Dense layers process user/item profiles
# 2. Merge and final Dense layer to obtain prediciton
class TwoTowerRecommender_type2(DeepLearningRecommender):

    RECOMMENDER_NAME = """TWO_TOWER_2"""

    def __init__(self, URM_train, num_users, num_items, layers=[10], reg_layers=[0], verbose = True):
        super().__init__(URM_train, verbose)
        layers[0] = int(layers[0]/2) # <- The first layer is split in two tower inputs at the beginning
        self.mlp_embedding_user = nn.Embedding(num_users, layers[0], device=self.device)
        self.mlp_embedding_item = nn.Embedding(num_items, layers[0], device=self.device) # <- It's possible to make the towers asymmetric! Mind the output dimension though

        self.mlp_layers_tower1 = nn.ModuleList([
            nn.Linear(
                layers[i-1],
                layers[i], bias=True, device=self.device
                ) for i in range(1, len(layers))
            ])
        
        self.mlp_layers_tower2 = nn.ModuleList([
            nn.Linear(
                layers[i-1],
                layers[i], bias=True, device=self.device
                ) for i in range(1, len(layers))
            ])
        
        for i, layer in enumerate(self.mlp_layers_tower1):
            nn.init.normal_(layer.weight)
            layer.bias.data.zero_()
            layer.weight_decay = reg_layers[i]

        for i, layer in enumerate(self.mlp_layers_tower2):
            nn.init.normal_(layer.weight)
            layer.bias.data.zero_()
            layer.weight_decay = reg_layers[i]

        self.prediction_layer = nn.Linear(layers[-1], 1, bias=True, device=self.device)
        nn.init.uniform_(self.prediction_layer.weight)
        self.prediction_layer.bias.data.zero_()
        self.to(self.device)

    def forward(self, user_input, item_input):
        mlp_user_latent = self.mlp_embedding_user(user_input.long().to(self.device))
        mlp_item_latent = self.mlp_embedding_item(item_input.long().to(self.device))

        mlp_user_vector = mlp_user_latent
        mlp_item_vector = mlp_item_latent

        for layer in self.mlp_layers_tower1:
            mlp_user_vector = torch.relu(layer(mlp_user_vector))

        for layer in self.mlp_layers_tower2:
            mlp_item_vector = torch.relu(layer(mlp_item_vector))

        predict_vector = mlp_user_vector * mlp_item_vector # <- Merge the tensors via element-wise multiplication
        prediction = torch.sigmoid(self.prediction_layer(predict_vector))
        return prediction

In [ ]:
# Train and test type 1
twotower_1 = TwoTowerRecommender_type1(URM_train, URM_train.shape[0], URM_train.shape[1], layers=[10,5,2,2], reg_layers=[0,0,0,0])

twotower_1.fit(epochs=100, batch_size=1024, learning_rate=0.01)

results_df, _ = evaluator_test.evaluateRecommender(twotower_1)

results_df

TWO_TOWER_1: URM Detected 76 ( 0.7%) items with no interactions.
TWO_TOWER_1: Epoch 0 finished. Loss: 1.423458218574524
TWO_TOWER_1: Epoch 1 finished. Loss: 1.334832787513733
TWO_TOWER_1: Epoch 2 finished. Loss: 1.2557604312896729
TWO_TOWER_1: Epoch 3 finished. Loss: 1.181513786315918
TWO_TOWER_1: Epoch 4 finished. Loss: 1.1167007684707642
TWO_TOWER_1: Epoch 5 finished. Loss: 1.0595446825027466
TWO_TOWER_1: Epoch 6 finished. Loss: 1.0078457593917847
TWO_TOWER_1: Epoch 7 finished. Loss: 0.959753155708313
TWO_TOWER_1: Epoch 8 finished. Loss: 0.9204481244087219
TWO_TOWER_1: Epoch 9 finished. Loss: 0.8844990730285645
TWO_TOWER_1: Epoch 10 finished. Loss: 0.8526425361633301
TWO_TOWER_1: Epoch 11 finished. Loss: 0.8253729939460754
TWO_TOWER_1: Epoch 12 finished. Loss: 0.8013153672218323
TWO_TOWER_1: Epoch 13 finished. Loss: 0.7806523442268372
TWO_TOWER_1: Epoch 14 finished. Loss: 0.763870894908905
TWO_TOWER_1: Epoch 15 finished. Loss: 0.7467600107192993
TWO_TOWER_1: Epoch 16 finished. Loss: 

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.129374,0.142682,0.068596,0.065975,0.071362,0.298375,0.114438,0.089655,0.598628,0.434521,...,0.998927,0.597985,0.998927,0.001697,4.478696,0.947075,0.008718,0.395494,2.764258,0.082079


In [15]:
# Train and test type 2
twotower_2 = TwoTowerRecommender_type2(URM_train, URM_train.shape[0], URM_train.shape[1], layers=[10,5,2,2], reg_layers=[0,0,0,0])

twotower_2.fit(epochs=100, batch_size=1024, learning_rate=0.01)

results_df, _ = evaluator_test.evaluateRecommender(twotower_2)

results_df

TWO_TOWER_2: URM Detected 76 ( 0.7%) items with no interactions.
TWO_TOWER_2: Epoch 0 finished. Loss: 0.7077656388282776
TWO_TOWER_2: Epoch 1 finished. Loss: 0.7031283974647522
TWO_TOWER_2: Epoch 2 finished. Loss: 0.6991095542907715
TWO_TOWER_2: Epoch 3 finished. Loss: 0.6958059668540955
TWO_TOWER_2: Epoch 4 finished. Loss: 0.6929289698600769
TWO_TOWER_2: Epoch 5 finished. Loss: 0.690422773361206
TWO_TOWER_2: Epoch 6 finished. Loss: 0.6882591247558594
TWO_TOWER_2: Epoch 7 finished. Loss: 0.6863564848899841
TWO_TOWER_2: Epoch 8 finished. Loss: 0.6846165657043457
TWO_TOWER_2: Epoch 9 finished. Loss: 0.6830786466598511
TWO_TOWER_2: Epoch 10 finished. Loss: 0.6816461086273193
TWO_TOWER_2: Epoch 11 finished. Loss: 0.6802965998649597
TWO_TOWER_2: Epoch 12 finished. Loss: 0.678999125957489
TWO_TOWER_2: Epoch 13 finished. Loss: 0.6777475476264954
TWO_TOWER_2: Epoch 14 finished. Loss: 0.676535964012146
TWO_TOWER_2: Epoch 15 finished. Loss: 0.6753504276275635
TWO_TOWER_2: Epoch 16 finished. Loss

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.000284,0.000284,0.000045,0.000096,0.000097,0.000906,0.000161,0.000077,0.002679,0.000933,...,0.998927,0.002676,0.998927,0.003194,4.063549,0.919136,0.016413,0.358834,0.008847,0.167405


## Graph Convolution

### LightGCN


In [ ]:
def computer(self):
        """
        propagate methods for lightGCN
        """
        users_emb = self.embedding_user.weight
        items_emb = self.embedding_item.weight
        all_emb = torch.cat([users_emb, items_emb])
        embs = [all_emb]
        if self.dropout_rate > 0.0:
            if self.training:
                g_dropped = self.__dropout(1 - self.dropout_rate)
            else:
                g_dropped = self.Graph
        else:
            g_dropped = self.Graph

        for layer in range(self.n_layers):
            all_emb = torch.sparse.mm(g_dropped, all_emb) # <- G * all_emb
            embs.append(all_emb) # <- Collect results
        embs = torch.stack(embs, dim=1)
        light_out = torch.mean(embs, dim=1) # <- Output = mean of collection
        users, items = torch.split(light_out, [self.n_users, self.n_items])
        return users, items

In [ ]:
def forward(self, users, items):
        # compute embedding
        all_users, all_items = self.computer()
        users_emb = all_users[users]
        items_emb = all_items[items]
        inner_pro = torch.mul(users_emb, items_emb)
        gamma = torch.sum(inner_pro, dim=1)
        return gamma

### GF-CF

In [17]:
from sklearn.utils.extmath import randomized_svd

def fit(self, alpha=1.0, num_factors=50, random_seed = None):
        self._print("Computing SVD decomposition of the normalized adjacency matrix...")

        self.alpha = alpha

        self.D_I = np.sqrt(np.array(self.URM_train.sum(axis = 0))).squeeze()
        self.D_I_inv = 1/(self.D_I + 1e-6)
        self.D_U_inv = 1/np.sqrt(np.array(self.URM_train.sum(axis = 1))).squeeze() + 1e-6

        self.D_I = sp.diags(self.D_I)
        self.D_I_inv = sp.diags(self.D_I_inv)
        self.D_U_inv = sp.diags(self.D_U_inv)

        self.R_tilde = self.D_U_inv.dot(self.URM_train).dot(self.D_I_inv)

        _, _, self.V = randomized_svd(self.R_tilde,
                                     n_components = num_factors,
                                     random_state = random_seed)

        self.D_I = sp.csr_matrix(self.D_I)
        self.D_I_inv = sp.csr_matrix(self.D_I_inv)